# BERT for multiclass text classification with PyTorch.

This is project for the Fellowship.AI program.

At first we do all the necessary imports and download transformers library.

In [23]:
import os
import nltk
import time
import torch
import string
import random
import datetime
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
!pip install transformers
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, AdamW

Here we download our train and test set and print its shape.

In [3]:
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space'
]

train= fetch_20newsgroups(subset="train" ,categories =categories) 
test= fetch_20newsgroups(subset="test" ,categories =categories)
X_train = train["data"]
X_test=test['data']
y_train = train["target"] 
y_test=test['target']

train_df=pd.DataFrame(X_train,columns=['mess'])
train_df['target']=y_train
print(train_df.shape)

test_df=pd.DataFrame(X_test,columns=['mess'])
test_df['target']=y_test
print(test_df.shape)

(2034, 2)
(1353, 2)


We want to clean our texts by discarding the punctuation and unnecessary stop words.

In [4]:
#making a function to process our data
def text_process(mess):
    no_punc=[c for c in mess if c not in string.punctuation]
    no_punc=''.join(no_punc)
    cleaned_mess=[word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
    return " ".join(cleaned_mess)

In [5]:
nltk.download('stopwords')

train_df['message']=train_df['mess'].apply(text_process)
#dropping our previous unprocessed column
train_df.drop('mess',axis=1,inplace=True)
train_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,target,message
0,1,rychfestivaledacuk R Hawkes Subject 3DS textur...
1,3,Subject Biblical Backing Koreshs 302 Tape Cite...
2,2,MarkPerewp201f208n103z1fidonetorg Subject Come...
3,0,dpwseicmuedu David Wood Subject Request Suppor...
4,2,prbaccessdigexcom Pat Subject Conference Manne...


In [6]:
test_df['message']=test_df['mess'].apply(text_process)
#dropping our previous unprocessed column
test_df.drop('mess',axis=1,inplace=True)
test_df.head()

,target,message
0,2,prbaccessdigexnet Pat Subject Near Miss Astero...
1,1,ing1023eeupacza ING1023 Subject Vatican librar...
2,1,Zheng Wang zw10andrewcmuedu Subject help Organ...
3,1,edimgwillardatlgaus Ed Pimentel Subject RFD co...
4,1,sternbrahmsudeledu Garland Stern Subject looki...


Small check on what GPU google colab is providing for us.

In [7]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


Next step would be downloading tokenizer for our BERT-large model and actually performing all the necessary preprocessing steps that BERT expects. The maximum number of tokens that BERT accepts is 512, but we will go even further and use 128, as we're facing the error using 512 value  related to memory allocation.

In [8]:
sentences = train_df["message"].values
labels = train_df["target"].values

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

input_ids = []
for sentence in sentences:
  encoded_sentence = tokenizer.encode(sentence, add_special_tokens=True)
  input_ids.append(encoded_sentence)

MAX_LEN = 128
print(f"Max number of tokens in sentences: {MAX_LEN}")

input_ids = pad_sequences(input_ids, value=0, padding='post', maxlen=MAX_LEN)

attention_masks = []
for encoded_sentence in input_ids:
  attention_mask = [int(token > 0) for token in encoded_sentence]
  attention_masks.append(attention_mask)

Loading BERT tokenizer...


Token indices sequence length is longer than the specified maximum sequence length for this model (1093 > 512). Running this sequence through the model will result in indexing errors


Max number of tokens in sentences: 128


In [9]:
# Use 90% for training and 10% for validation.
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, 
                                                            random_state=2021, test_size=0.1)

# Do the same for the masks.

train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2021, test_size=0.1)



# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(val_inputs)
print(train_inputs.size(), validation_inputs.size())

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(val_labels)
print(train_labels.size(), validation_labels.size())

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
print(train_masks.size(), validation_masks.size())

torch.Size([1830, 128]) torch.Size([204, 128])
torch.Size([1830]) torch.Size([204])
torch.Size([1830, 128]) torch.Size([204, 128])


In [10]:
batch_size = 8

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [11]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-large-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(categories), # The number of output labels--2 for binary classification.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [12]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Some helper functions.

In [13]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [14]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode.
    # `dropout` and `batchnorm` layers behave differently during training vs. test
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print(f'  Batch {step:>5}  of  {len(train_dataloader):>5}.    Elapsed: {elapsed}.')

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print(f"  Average training loss: {avg_train_loss:.2f}")
    print(f"  Training epcoh took: {format_time(time.time() - t0)}")
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.

        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print(f"  Accuracy: {eval_accuracy/nb_eval_steps:.2f}")
    print(f"  Validation took: {format_time(time.time() - t0)}")

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    50  of    229.    Elapsed: 0:00:20.
  Batch   100  of    229.    Elapsed: 0:00:40.
  Batch   150  of    229.    Elapsed: 0:01:00.
  Batch   200  of    229.    Elapsed: 0:01:20.

  Average training loss: 0.64
  Training epcoh took: 0:01:31

Running Validation...
  Accuracy: 0.80
  Validation took: 0:00:03

======== Epoch 2 / 4 ========
Training...
  Batch    50  of    229.    Elapsed: 0:00:20.
  Batch   100  of    229.    Elapsed: 0:00:40.
  Batch   150  of    229.    Elapsed: 0:01:00.
  Batch   200  of    229.    Elapsed: 0:01:19.

  Average training loss: 0.29
  Training epcoh took: 0:01:31

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:03

======== Epoch 3 / 4 ========
Training...
  Batch    50  of    229.    Elapsed: 0:00:20.
  Batch   100  of    229.    Elapsed: 0:00:40.
  Batch   150  of    229.    Elapsed: 0:00:59.
  Batch   200  of    229.    Elapsed: 0:01:19.

  Average training loss: 0.13
  Training epcoh t

In [15]:
sentences = test_df["message"].values
labels = test_df["target"].values

input_ids = []
for sentence in sentences:
  encoded_sentence = tokenizer.encode(sentence, add_special_tokens=True)
  input_ids.append(encoded_sentence)

input_ids = pad_sequences(input_ids, value=0, padding='post', maxlen=MAX_LEN)

attention_masks = []
for encoded_sentence in input_ids:
  attention_mask = [int(token > 0) for token in encoded_sentence]
  attention_masks.append(attention_mask)

In [16]:
input_ids = torch.tensor(input_ids)
labels = torch.tensor(labels)
attention_masks = torch.tensor(attention_masks)

# Create the DataLoader for our test set.
test_data = TensorDataset(input_ids, attention_masks, labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [17]:
# Prediction on test set

print(f'Predicting labels for {len(input_ids)} test sentences...')

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 1353 test sentences...
    DONE.


The final step would be printing some performance measures of our model on the test set.

In [18]:
predictions = np.concatenate(predictions)
true_labels = np.concatenate(true_labels)
flat_accuracy(predictions, true_labels)

0.844789356984479

In [22]:
print(classification_report(np.argmax(predictions, axis=1), true_labels))

              precision    recall  f1-score   support

           0       0.76      0.73      0.75       335
           1       0.94      0.96      0.95       382
           2       0.93      0.91      0.92       403
           3       0.66      0.71      0.69       233

    accuracy                           0.84      1353
   macro avg       0.82      0.83      0.83      1353
weighted avg       0.85      0.84      0.85      1353



On classes 1 and 2 we were able to achieve the f1-score comparable with ones presented on kaggle, but we can see clear problem with classes 0 and 3. The reason for that might be due to the having not enough training examples corresponding to that class, as presented below.

In [26]:
train_df['target'].value_counts()

2    593
1    584
0    480
3    377
Name: target, dtype: int64